In [6]:
import numpy as np
import sklearn

from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import ShuffleSplit, GridSearchCV, train_test_split
from sklearn import svm, metrics
from sklearn.cluster import KMeans

In [5]:
# Importamos los datos

X=np.load("./datos_mfcc_pca99_X.npy")
y=np.load("./datos_mfcc_pca99_y.npy")

In [8]:
#Reducimos el conjunto de entrenamiento al 50% usando un método no supervisado. En este caso, un C-means con el que pretendemos
# quedarnos con los centros de los clusters que aparezcan, que serán los puntos más representativos.

c = 6             # número de clases
nc = 120          # muestras por clase
n= 720           # número de muestras totales

X_reduc=[]
y_reduc=np.int16(np.kron(np.arange(c),np.ones(nc//2)))

for i in range(0,n,nc):
    k_means=KMeans(n_clusters=nc//2)
    k_means.fit(X[i:(i+nc)])
    centros=k_means.cluster_centers_
    X_reduc.append(centros)

In [13]:
print(X_reduc[0].shape)
# Reshapeamos las dimensiones de X_reduc para tenerlo todo en una matriz dónde cada fila será una muestra y cada columna una 
# característica. La matriz estará ordenada por clases. 
# Para reshapear, antes habrá que convertir a array.


X_reduc=np.array(X_reduc).reshape((360,464))
print(X_reduc.shape)

(60, 464)
(360, 464)


In [18]:
# Ahora ya podemos dividir nuestro conjunto reducido en entrenamiento y test, y probar algunos métodos Kernel.

Xtr, Xts, ytr, yts = train_test_split(X_reduc, y_reduc, test_size=0.3, shuffle=True, random_state=0)


In [23]:
# Aplicamos directamente un clasificador SVC con los parámetros por defecto y calculamos algunas métricas para ver cómo funciona

clf=svm.SVC(verbose=True)
clf.fit(Xtr,ytr)

score_tr = clf.score(Xtr,ytr)
score_ts = clf.score(Xts,yts)
preds_train=clf.predict(Xtr)
preds_test = clf.predict(Xts)

print('')
print('OA train %0.2f' % score_tr)
print('Kappa train %0.2f' % metrics.cohen_kappa_score(ytr,preds_train))
print('OA test %0.2f' % score_ts)
print('Kappa test %0.2f' % metrics.cohen_kappa_score(yts,preds_test))


[LibSVM]
OA train 0.92
Kappa train 0.90
OA test 0.71
Kappa test 0.66


In [26]:
# Matriz de confusión

CM=metrics.confusion_matrix(yts,preds_test)
print(CM)

[[12  0  1  0  1  3]
 [ 2 12  0  1  0  5]
 [ 2  0 15  1  0  2]
 [ 2  0  3 17  0  0]
 [ 3  2  1  0 12  0]
 [ 0  0  0  1  1  9]]


Observamos que con los parámetros por defecto, obtenemos unos resultados bastante buenos en entrenamiento y un acierto bastante decente en test. 

Además, vemos que la matriz de confusión está bastante balanceada, por lo que tiene buena pinta este clasificador para nuestros datos.

Veamos ahora qué ocurre si hacemos una búsqueda por cross-validation de los parámetros ideales. Queremos intentar mejorar lo que ya tenemos.

In [24]:
# Los parámetros por defecto usados han sido 

clf.get_params() 

# dónde gamma= 'scale' viene definida por la siguiente fórmula: gamma = 1/(464*Xtr[2].var())

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': True}

In [40]:
# Buscamos los parámetros óptimos con GridSearchCV

gammas = np.logspace(-9, 7, 10)
Cs = np.logspace(-9, 7, 10)  
tuned_parameters = { 'gamma': gammas,'C': Cs} 

clf2 = GridSearchCV(svm.SVC(kernel='rbf'), tuned_parameters, cv=5,n_jobs=-1,verbose=0)
clf2.fit(Xtr,ytr)
clf2=clf2.best_estimator_

score_tr = clf2.score(Xtr,ytr)
score_ts = clf2.score(Xts,yts)
preds_train=clf2.predict(Xtr)
preds_test = clf2.predict(Xts)

print('')
print('OA train %0.2f' % score_tr)
print('Kappa train %0.2f' % metrics.cohen_kappa_score(ytr,preds_train))
print('OA test %0.2f' % score_ts)
print('Kappa test %0.2f' % metrics.cohen_kappa_score(yts,preds_test))


OA train 0.93
Kappa train 0.92
OA test 0.77
Kappa test 0.72


In [41]:
clf2.get_params()

{'C': 2782.5594022071145,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 1e-09,
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [42]:
# Matriz de confusión

CM=metrics.confusion_matrix(yts,preds_test)
print(CM)

[[12  1  0  1  1  2]
 [ 2 12  0  1  1  4]
 [ 2  0 15  1  0  2]
 [ 2  0  1 19  0  0]
 [ 3  0  0  0 15  0]
 [ 0  0  0  1  0 10]]


En general, vemos como al buscar los parámetros óptimos de la función Kernel (='rbf'), los resultados mejoran en todos los aspectos. Obtenemos un mayor acierto tanto en entrenamiento como en test y están balanceados, por lo que en principio no sospecharíamos de sobreajuste. 

Además, al ver la forma de la matriz de confusión volvemos a ver lo mismo, los resultados están muy balanceados.